In [1]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig
from trl import SFTTrainer
from tqdm import tqdm
from lazypredict.Supervised import LazyClassifier

df = pd.read_csv("edos_labelled_aggregated.csv")
df = df[df['label_vector'] != 'none']


train_df, dev_df, test_df = df[df['split'] == 'train'], df[df['split'] == 'dev'], df[df['split'] == 'test']

print(f"train: {train_df.shape[0]}, dev:{dev_df.shape[0]}, test:{test_df.shape[0]}")

# Define the prompt template
prompt_template = """Fine-grained Vector of Sexism: for posts which are sexist, an 11-class classification where systems have to predict one of 11 fine-grained vectors.

Given a post determine the post is belong to which class:
1.1 threats of harm
1.2 incitement and encouragement of harm
2.1 descriptive attacks
2.2 aggressive and emotive attacks
2.3 dehumanising attacks & overt sexual objectification
3.1 casual use of gendered slurs, profanities, and insults
3.2 immutable gender differences and gender stereotypes
3.3 backhanded gendered compliments
3.4 condescending explanations or unwelcome advice
4.1 supporting mistreatment of individual women
4.2 supporting systemic discrimination against women as a group

### Post: 
{POST}
### Class: """

column='label_vector'

train: 3398, dev:486, test:970


In [12]:
llm_path = "task_c_llm"

tokenizer = AutoTokenizer.from_pretrained(llm_path, padding_side="left")

tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model with 4-bit precision
finetuned_model = AutoModelForCausalLM.from_pretrained(llm_path, quantization_config=quant_config, device_map={"": 0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
# len(tokenizer.encode("1. threats, plans to harm and incitement"))

In [14]:
from torch.utils.data import DataLoader

class EDOSDataset(Dataset):
    def __init__(self, df, prompt_template, column):
        self.texts = df['text'].tolist()
        self.labels = df[column].tolist()
        self.prompt_template=prompt_template

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idxs):
        inputs, inputs_labels = [], []
        for idx in idxs:
            
            inputs.append(self.prompt_template.replace("{POST}", self.texts[idx]))
            inputs_labels.append(self.labels[idx])
        
        return {"inputs":inputs, "labels": inputs_labels}
    
def make_the_generations(model, tokenizer, data_loader):
    gen_texts, labels = [], []
    
    for batch in tqdm(data_loader):
        input_data = batch['inputs']
        labels += batch['labels']
        tokenized_input_data = tokenizer(input_data, padding=True, max_length=512, truncation=True, return_tensors="pt").to("cuda:0")
        # print(tokenized_input_data)
        outputs = finetuned_model.generate(
            **tokenized_input_data,
            pad_token_id= tokenizer.eos_token_id,
            max_new_tokens=20,
            do_sample=True
        )
        generated_texts = [tokenizer.decode(outputs[idx], skip_special_tokens=True)[len(input_data[idx]):]
                          for idx in range(len(outputs))]
        gen_texts += generated_texts
    return gen_texts, labels

In [15]:
batch_size = 64

train_data = EDOSDataset(df=train_df, prompt_template=prompt_template, column=column)
train_dataloader =  DataLoader(train_data, batch_size=batch_size, shuffle=False)
train_texts, train_labels = make_the_generations(finetuned_model, tokenizer, train_dataloader)

100%|██████████| 54/54 [01:57<00:00,  2.17s/it]


In [16]:
dev_data = EDOSDataset(df=dev_df, prompt_template=prompt_template, column=column)
dev_dataloader =  DataLoader(dev_data, batch_size=batch_size, shuffle=False)
dev_texts, dev_labels = make_the_generations(finetuned_model, tokenizer, dev_dataloader)

100%|██████████| 8/8 [00:17<00:00,  2.13s/it]


In [17]:
test_data = EDOSDataset(df=test_df, prompt_template=prompt_template, column=column)
test_dataloader =  DataLoader(test_data, batch_size=batch_size, shuffle=False)
test_texts, test_labels = make_the_generations(finetuned_model, tokenizer, test_dataloader)

100%|██████████| 16/16 [00:33<00:00,  2.12s/it]


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV, LogisticRegressionCV, ElasticNetCV
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.calibration import CalibratedClassifierCV

vectorizer_1 = CountVectorizer(ngram_range=(1,4), lowercase=True)
vectorizer_2 = TfidfVectorizer( ngram_range=(1,5), 
                                     lowercase=True, 
                                     sublinear_tf=True, 
                                     use_idf=True)
features = FeatureUnion([
    ("count-vec", vectorizer_1),
    ("tfidf", vectorizer_2),
])

class_mapper = Pipeline (
    steps=[
        ("Vectorizer", features),
        # ("TruncatedSVD", TruncatedSVD(n_components=600)),
        ('Classifier', LogisticRegression())
])

# class_mapper.fit(train_texts+train_labels+dev_texts, train_labels+train_labels+dev_labels)
# class_mapper.fit(train_labels, train_labels)
# class_mapper.fit(train_texts+train_labels, train_labels+train_labels)
class_mapper.fit(train_texts+train_labels+train_df['text'].tolist(), 
                 train_labels+train_labels+train_df[column].tolist())
# +train_texts

Pipeline(steps=[('Vectorizer',
                 FeatureUnion(transformer_list=[('count-vec',
                                                 CountVectorizer(ngram_range=(1,
                                                                              4))),
                                                ('tfidf',
                                                 TfidfVectorizer(ngram_range=(1,
                                                                              5),
                                                                 sublinear_tf=True))])),
                ('Classifier', LogisticRegression())])

In [27]:
train_predict = class_mapper.predict(train_texts)
dev_predict = class_mapper.predict(dev_texts)
test_predict = class_mapper.predict(test_texts)

print("TRAIN"+"-"*150)
print(classification_report(train_labels, train_predict, digits=4))
print("DEV"+"-"*150)
print(classification_report(dev_labels, dev_predict, digits=4))
print("TEST"+"-"*150)
print(classification_report(test_labels, test_predict, digits=4))

TRAIN------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                 precision    recall  f1-score   support

                                            1.1 threats of harm     0.9038    0.8393    0.8704        56
                       1.2 incitement and encouragement of harm     0.9669    0.9213    0.9435       254
                                        2.1 descriptive attacks     0.8899    0.9247    0.9070       717
                             2.2 aggressive and emotive attacks     0.8784    0.9123    0.8950       673
        2.3 dehumanising attacks & overt sexual objectification     0.9239    0.9100    0.9169       200
     3.1 casual use of gendered slurs, profanities, and insults     0.9051    0.9137    0.9094       637
        3.2 immutable gender differences and gender stereotypes     0.9102    0.8753    0.8924       417
   

In [28]:
from sentence_transformers import SentenceTransformer

sbert = SentenceTransformer("sentence-transformers/nli-mpnet-base-v2")

train_texts_vec = sbert.encode(train_texts+train_df['text'].tolist(), show_progress_bar=True)
dev_texts_vec = sbert.encode(dev_texts, show_progress_bar=True)
test_texts_vec = sbert.encode(test_texts, show_progress_bar=True)

Batches:   0%|          | 0/213 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

In [29]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# clf = MLPClassifier(hidden_layer_sizes=(100,100),max_iter=1000)
clf=LogisticRegression()
clf.fit(train_texts_vec, train_labels+train_labels)

train_predict = clf.predict(train_texts_vec)
dev_predict = clf.predict(dev_texts_vec)
test_predict = clf.predict(test_texts_vec)

print("TRAIN"+"-"*150)
print(classification_report(train_labels+train_labels, train_predict, digits=4))
print("DEV"+"-"*150)
print(classification_report(dev_labels, dev_predict, digits=4))
print("TEST"+"-"*150)
print(classification_report(test_labels, test_predict, digits=4))

TRAIN------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                 precision    recall  f1-score   support

                                            1.1 threats of harm     0.8721    0.6696    0.7576       112
                       1.2 incitement and encouragement of harm     0.8577    0.8071    0.8316       508
                                        2.1 descriptive attacks     0.7277    0.8068    0.7652      1434
                             2.2 aggressive and emotive attacks     0.7234    0.7831    0.7521      1346
        2.3 dehumanising attacks & overt sexual objectification     0.7934    0.6625    0.7221       400
     3.1 casual use of gendered slurs, profanities, and insults     0.7789    0.7771    0.7780      1274
        3.2 immutable gender differences and gender stereotypes     0.7601    0.7410    0.7505       834
   